# **1 - Tarefa e Dados**
Descreva a tarefa escolhida e os dados. Escreva código que leia os dados e calcule e imprima quantas instâncias os dados têm. Também, seu código deve calcular a média de tokens por instância, isto é, quantos tokens, na média cada documento do seu conjunto de dados possui. Imprima esse único número.


A corpus escolhido foi o UTL que é um corpus com críticas de filmes e apps coletadas automaticamente de sites. As classes são: positiva ou negativa. Assim o usuário pode ter gostado ou não gostado do produto. Referência: https://github.com/RogerFig/UTLCorpus 


# montando o drive do google onde o corpus se encontra
from google.colab import drive
drive.mount('/content/drive')

# conferir que o Google Drive está devidamente montado (deve aparecer a pasta 'drive')
%ls -la

# Entrando no diretório
%cd drive/My Drive/Colab Notebooks/NLP/trabalho

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Herica\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import pandas as pd
import numpy as np

In [19]:
%matplotlib inline
#nltk.download('punkt')
nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Herica\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# abrindo o arquivo.
df_util = pd.read_csv(r'C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\UTLCorpus.csv')
df_util.head(3)

,Unnamed: 0,domain,object,author,text,likes,unlikes,stars,date,collect_date,replies,favorite,want_see,recommend,see,internal_id
0,0,movies,007-cassino-royale-t23,0,Um dos melhores do 007,0,0,3.5,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,0
1,1,movies,007-cassino-royale-t23,1,"assisti só pela metade..do meio ao fim, o film...",0,0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,1
2,2,movies,007-cassino-royale-t23,2,foi um dos filmes mais violentos q já vi. mas ...,0,0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0,2


# Analisando os dados

In [21]:
# Verificando quais os tipos da feature domain
df_util.domain.unique()

array(['movies', 'apps'], dtype=object)

In [22]:
qtde_moveis = len(df_util[df_util['domain'] == 'movies'])
qtde_apps = len(df_util[df_util['domain'] == 'apps'])

print ('qtde de registros do tipo movies', qtde_moveis)
print ('qtde de registros do tipo app', qtde_apps)

qtde de registros do tipo movies 1839851
qtde de registros do tipo app 1041738


In [23]:
# Selecionando somente o registros do tipo movies
df_movies = df_util[df_util['domain'] == 'movies']

## Verificando e deletando registros nulos

In [24]:
df_movies.isnull().sum()

Unnamed: 0            0
domain                0
object                0
author                0
text                956
likes                 0
unlikes               0
stars                 0
date                  0
collect_date          0
replies               0
favorite              0
want_see              0
recommend       1839851
see                   0
internal_id           0
dtype: int64

In [25]:
# deletando os registros nulos
qtde_anterior_nulos = df_movies.shape[0]
df_movies.dropna(subset=['text'], inplace = True)
qtde_pos_exclusao_nulos = df_movies.shape[0]
print ("Antes:", qtde_anterior_nulos, 'Depois:', qtde_pos_exclusao_nulos, 'Diferença:', qtde_anterior_nulos-qtde_pos_exclusao_nulos)

Antes: 1839851 Depois: 1838895 Diferença: 956


In [26]:
df_movies['domain'].value_counts()

movies    1838895
Name: domain, dtype: int64

## Avaliando as features like e unlikes

In [27]:
df_movies['likes'].describe()

count    1.838895e+06
mean     1.723991e+00
std      4.488318e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      4.130000e+02
Name: likes, dtype: float64

In [28]:
df_movies['likes'].value_counts()

0      887802
1      372855
2      208220
3      121399
4       74471
5       47654
6       31812
7       21632
8       15425
9       11130
10       7826
11       6166
12       4680
13       3641
14       3078
15       2323
16       2064
17       1639
18       1404
19       1208
20       1100
21        911
22        835
23        669
24        648
25        592
27        491
26        473
29        407
28        387
        ...  
329         1
221         1
171         1
217         1
183         1
353         1
142         1
146         1
149         1
152         1
169         1
173         1
177         1
179         1
180         1
181         1
187         1
215         1
195         1
196         1
197         1
198         1
199         1
202         1
203         1
204         1
209         1
212         1
213         1
413         1
Name: likes, Length: 233, dtype: int64

In [29]:
df_movies['unlikes'].describe()

count    1838895.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
Name: unlikes, dtype: float64

In [30]:
df_movies['unlikes'].value_counts()

0    1838895
Name: unlikes, dtype: int64

In [31]:
# De acordo com a avaliação das features likes e unlikes, todos os registros de unlikes são iguais a zero, ou seja, não há registros da quantidade de registros de pessoas que deram unlike nos filmes, logo a feature não se´ra utilizada.
print ('qtde de registros da base:', len(df_movies), '\nqtde de registros de likes igual a zero: ', len(df_movies['likes'][df_movies['likes'] == 0]))

qtde de registros da base: 1838895 
qtde de registros de likes igual a zero:  887802


In [32]:
df_movies = df_movies[['domain', 'object', 'author', 'text', 'likes',
       'stars', 'date', 'collect_date', 'replies', 'favorite', 'want_see',
       'recommend', 'see']]
df_movies.head(3)

,domain,object,author,text,likes,stars,date,collect_date,replies,favorite,want_see,recommend,see
0,movies,007-cassino-royale-t23,0,Um dos melhores do 007,0,3.5,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0
1,movies,007-cassino-royale-t23,1,"assisti só pela metade..do meio ao fim, o film...",0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0
2,movies,007-cassino-royale-t23,2,foi um dos filmes mais violentos q já vi. mas ...,0,4.0,8 anos atrás,20-04-2019 06:20,0.0,0.0,0.0,NaN,1.0


## Analisando a feature recommend

In [33]:
df_movies['recommend'].value_counts()

Series([], Name: recommend, dtype: int64)

In [34]:
df_movies['recommend'].unique()

array([nan])

In [35]:
# como a feature não possui valores, não será utilizada
df_movies = df_movies[['domain', 'object', 'author', 'text', 'likes', 'stars', 'date',
      'replies', 'favorite', 'want_see', 'see']]

## Analisando a feature want_see

In [36]:
df_movies['want_see'].unique()

array([0., 1.])

In [37]:
df_movies['want_see'].value_counts()

0.0    1814467
1.0      24428
Name: want_see, dtype: int64

In [38]:
df_movies.columns

Index(['domain', 'object', 'author', 'text', 'likes', 'stars', 'date',
       'replies', 'favorite', 'want_see', 'see'],
      dtype='object')

In [39]:
df_movies[['object','text','want_see','stars']][df_movies['want_see'] == 0 ]

,object,text,want_see,stars
0,007-cassino-royale-t23,Um dos melhores do 007,0.0,3.5
1,007-cassino-royale-t23,"assisti só pela metade..do meio ao fim, o film...",0.0,4.0
2,007-cassino-royale-t23,foi um dos filmes mais violentos q já vi. mas ...,0.0,4.0
3,007-cassino-royale-t23,Adoroo esse filme!,0.0,0.0
4,007-cassino-royale-t23,O Daniel Crieg tem carisma zero como 007...,0.0,4.0
5,007-cassino-royale-t23,um dos melhores de 007,0.0,4.0
6,007-cassino-royale-t23,"Nunca pensei que diria isso em um filme 007, m...",0.0,2.5
7,007-cassino-royale-t23,preciso rever,0.0,5.0
8,007-cassino-royale-t23,Adorei... Nossa esse Daniel Craig é UM HOMEM...,0.0,0.0
9,007-cassino-royale-t23,"Divertido, mas achei no inicio um pouco entedi...",0.0,3.0


## Analisando a feature see

In [40]:
df_movies['see'].unique()

array([1., 0.])

### Avaliando a feature stars

In [41]:
df_movies['stars'].describe()

count    1.838895e+06
mean     3.307427e+00
std      1.804395e+00
min      0.000000e+00
25%      2.500000e+00
50%      4.000000e+00
75%      5.000000e+00
max      5.000000e+00
Name: stars, dtype: float64

In [42]:
df_movies['stars'].value_counts()

5.0    505736
0.0    328620
4.0    326632
4.5    219635
3.5    158253
3.0    139654
2.5     54829
2.0     46486
0.5     22826
1.0     20391
1.5     15833
Name: stars, dtype: int64

In [43]:
# Definindo as faixas a serem criadas
intervals = pd.qcut(df_movies['stars'], 3).astype(str).unique()
intervals

array(['(3.0, 4.5]', '(-0.001, 3.0]', '(4.5, 5.0]'], dtype=object)

In [44]:
# Criando três faixas para a feature star
df_movies['stars_faixas'] = pd.qcut(df_movies['stars'], 3).astype(str).map( {
                            intervals[0]: 1, #'(-0.001, 3.0]'
                            intervals[1]: 0, #'(3.0, 4.5]'
                            intervals[2]: 2  # '(4.5, 5.0]'
                            } ).astype(str)
df_movies[['stars','stars_faixas']].head(3)

,stars,stars_faixas
0,3.5,1
1,4.0,1
2,4.0,1


In [45]:
print ('Qtde de filmes por faixa\n')
df_movies['stars_faixas'].value_counts()

Qtde de filmes por faixa



1    704520
0    628639
2    505736
Name: stars_faixas, dtype: int64

## Analisando a feature replies

In [46]:
df_movies['replies'].unique()

array([  0.,   2.,   1.,   4.,   3.,   8.,   6.,   7.,   5.,  11.,  12.,
        16.,  10.,   9.,  38.,  19.,  26.,  33.,  14.,  17.,  53.,  25.,
        21.,  32.,  13.,  15.,  22.,  18.,  20.,  71.,  30.,  28.,  29.,
        37.,  24.,  31.,  23.,  59.,  49.,  27.,  41.,  35.,  54.,  34.,
        55.,  42.,  40., 142.,  56.,  36.,  83.,  51.,  50.,  48.,  39.])

In [47]:
df_movies['replies'].value_counts()

0.0      1644092
1.0       136160
2.0        34546
3.0        11680
4.0         5250
5.0         2618
6.0         1479
7.0          871
8.0          584
9.0          410
10.0         257
11.0         185
12.0         130
13.0         118
14.0          95
15.0          78
16.0          54
17.0          49
19.0          31
18.0          27
20.0          25
22.0          17
23.0          16
21.0          14
25.0          12
30.0          11
24.0          10
27.0           9
33.0           8
26.0           8
29.0           5
28.0           4
38.0           4
40.0           4
35.0           3
36.0           3
42.0           3
31.0           3
53.0           3
37.0           2
32.0           2
34.0           2
51.0           1
142.0          1
83.0           1
71.0           1
59.0           1
39.0           1
56.0           1
50.0           1
55.0           1
41.0           1
54.0           1
49.0           1
48.0           1
Name: replies, dtype: int64

# DESCRIÇÃO DAS FEATURES QUE SERÃO UTILIZADAS NO TRABALHO
author -> author da avaliação

domanin -> domínio: filmes avaliado

like -> qtde de links dada pelo author para o filme

see -> se o filme foi ou não visto pelo author ( 0 ou 1)

want_see -> se o author quer ou não assistir o filme (0 ou 1)

stars -> qtde de estrelas dado pelo author para o filme

replies -> qtde de replicas que o comentário recebeu

date -> data da avaliação

In [48]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1838895 entries, 0 to 1839850
Data columns (total 12 columns):
domain          object
object          object
author          int64
text            object
likes           int64
stars           float64
date            object
replies         float64
favorite        float64
want_see        float64
see             float64
stars_faixas    object
dtypes: float64(5), int64(2), object(5)
memory usage: 182.4+ MB


# Verificando a existencia de registros nulos

In [49]:
df_movies.isnull().sum()

domain          0
object          0
author          0
text            0
likes           0
stars           0
date            0
replies         0
favorite        0
want_see        0
see             0
stars_faixas    0
dtype: int64

In [50]:
qtde_antes = (df_movies.shape[0])

In [51]:
# Deletando os registros texto nulos
df_movies.dropna(subset=['text'], inplace = True)

In [52]:
# Validando a exclusão
qtde_depois = (df_movies.shape[0])
print ('Qtde de registros excluidos', qtde_antes - qtde_depois)

Qtde de registros excluidos 0


## Verificando registros duplicados do par object x author


In [53]:
# encontrando os registros duplicados no author, ou seja, mais de uma avaliadação do author para o filme
df_duplicado = df_movies.groupby(['author','object']).size().reset_index(name='count')
df_duplicado.query('count > 1')

,author,object,count
4,0,127-horas-t24843,2
5,0,2001-uma-odisseia-no-espaco-t1422,3
7,0,21-gramas-t4077,2
15,0,a-arvore-da-vida-t14320,4
34,0,a-era-do-gelo-2-t1232,2
36,0,a-escolha-de-sofia-t4156,2
43,0,a-forma-da-agua-t203110,2
44,0,a-fraternidade-e-vermelha-t4179,2
49,0,a-grande-beleza-t62477,3
53,0,a-igualdade-e-branca-t2053,2


In [54]:
# conforme observado acima, há repetição de registros
# visualizando um dos registros
df_movies[df_movies['object'] == 'cisne-negro-t16349']

,domain,object,author,text,likes,stars,date,replies,favorite,want_see,see,stars_faixas
464338,movies,cisne-negro-t16349,4288,no filme temos a visão dela do que acontece......,0,5.0,8 anos atrás,0.0,1.0,0.0,1.0,2
464339,movies,cisne-negro-t16349,29909,"Achei genial a abordagem psicológica do filme,...",0,4.5,8 anos atrás,0.0,1.0,0.0,1.0,1
464340,movies,cisne-negro-t16349,42302,Filme sensacional!!Superou minhas expectativas...,0,5.0,8 anos atrás,0.0,0.0,0.0,1.0,2
464341,movies,cisne-negro-t16349,10376,Perturbador e Perfeito!,0,0.0,8 anos atrás,0.0,0.0,0.0,1.0,0
464342,movies,cisne-negro-t16349,74353,"Foda a garota ficar paranóica, ainda mais quan...",0,4.5,8 anos atrás,0.0,1.0,0.0,1.0,1
464343,movies,cisne-negro-t16349,10145,O filme é tão...'perturbador' que consegue nos...,0,4.5,8 anos atrás,0.0,0.0,0.0,1.0,1
464344,movies,cisne-negro-t16349,74354,"Concordo com a opinião abaixo, da Marianne, o ...",0,5.0,8 anos atrás,0.0,1.0,0.0,1.0,2
464345,movies,cisne-negro-t16349,22391,Um filme que lembra que o Cinema é também uma ...,0,5.0,8 anos atrás,0.0,1.0,0.0,1.0,2
464346,movies,cisne-negro-t16349,38155,Simplesmente S-E-N-S-A-C-I-O-N-A-L !,0,5.0,8 anos atrás,0.0,1.0,0.0,1.0,2
464347,movies,cisne-negro-t16349,7363,"Porra, média tá altíssima, hein.espero que iss...",0,4.0,8 anos atrás,0.0,0.0,0.0,1.0,1


In [55]:
# verificando alguns dos valores duplicados de like de um authos/object
#df_util.query('author == 0 and object == "127-horas-t24843"') # registros iguais
#df_util.query('author == 0 and object == "2001-uma-odisseia-no-espaco-t1422"') # registros de data diferente e com likes diferentes
df_movies.query('author == 0 and object == "a-arvore-da-vida-t14320"') # possui registros de data diferentes e likes diferentes. likes:0,4,5,5

,domain,object,author,text,likes,stars,date,replies,favorite,want_see,see,stars_faixas
31951,movies,a-arvore-da-vida-t14320,0,Isso aqui tá bombando em spoilers.,0,5.0,7 anos atrás,0.0,1.0,0.0,1.0,2
31991,movies,a-arvore-da-vida-t14320,0,"É o filme do Malick que mais me tocou, é o fil...",5,5.0,7 anos atrás,7.0,1.0,0.0,1.0,2
33686,movies,a-arvore-da-vida-t14320,0,Cinema para a eternidade.,4,5.0,6 anos atrás,0.0,1.0,0.0,1.0,2
34011,movies,a-arvore-da-vida-t14320,0,Eu jamais entenderei o porque que as pessoas a...,5,5.0,5 anos atrás,1.0,1.0,0.0,1.0,2


In [56]:
# Eliminando os registros com repetição, mantendo os que possuem a maior qtde de likes
qtde_anterior_duplicados = df_movies.shape[0]

df_movies = df_movies.sort_values(["author", "object", "likes"], ascending=[True,True,True])
df_movies.drop_duplicates(subset=['author', 'object'], keep='last', inplace = True)


qtde_pos_exclusao_duplicados = df_movies.shape[0]
print ('antes:', qtde_anterior_duplicados, 'depois:', qtde_pos_exclusao_duplicados, 'diferença:', qtde_anterior_duplicados-qtde_pos_exclusao_duplicados)

antes: 1838895 depois: 1736231 diferença: 102664


In [57]:
# validando a exclusão das duplicatas
#df_util.query('author == 0 and object == "127-horas-t24843"') # registros iguais
#df_util.query('author == 0 and object == "2001-uma-odisseia-no-espaco-t1422"') # registros de data diferente e com likes diferentes
df_movies.query('author == 0 and object == "a-arvore-da-vida-t14320"') # possui registros de data diferentes e likes diferentes. likes:0,4,5,5

,domain,object,author,text,likes,stars,date,replies,favorite,want_see,see,stars_faixas
34011,movies,a-arvore-da-vida-t14320,0,Eu jamais entenderei o porque que as pessoas a...,5,5.0,5 anos atrás,1.0,1.0,0.0,1.0,2


In [58]:
# contado e mostrando quais são os filmes únicos
registros_unicos = df_movies['object'].unique()
print ('qtde de registros na base',len(df_movies), '\nqtde de registros:', len(registros_unicos), '\nvalores unicos:', registros_unicos)

qtde de registros na base 1736231 
qtde de registros: 4203 
valores unicos: ['007-cassino-royale-t23' '007-operacao-skyfall-t31713'
 '10-coisas-que-eu-odeio-em-voce-t1964' ... 'um-caso-de-familia-t202015'
 'um-homem-e-sua-paixao-t108598' 'voce-nunca-mais-me-vera-t101629']


In [59]:
df_movies.head(5)

,domain,object,author,text,likes,stars,date,replies,favorite,want_see,see,stars_faixas
0,movies,007-cassino-royale-t23,0,Um dos melhores do 007,0,3.5,8 anos atrás,0.0,0.0,0.0,1.0,1
2489,movies,007-operacao-skyfall-t31713,0,Javier Bardem caricato. Daniel Craig ruim como...,0,3.5,6 anos atrás,2.0,0.0,0.0,1.0,1
6141,movies,10-coisas-que-eu-odeio-em-voce-t1964,0,ADORO!!,0,4.5,8 anos atrás,0.0,0.0,0.0,1.0,1
7108,movies,12-anos-de-escravidao-t43448,0,Existem certos filmes que funcionam como ápice...,2,5.0,5 anos atrás,0.0,1.0,0.0,1.0,2
14078,movies,127-horas-t24843,0,"O filme é do James Franco, sem dúvidas. Se não...",0,4.5,8 anos atrás,0.0,0.0,0.0,1.0,1


# Trabalhando os dados

In [60]:
# Fazendo uma cópia dos dados, e selecionando as features que serão utilizadas
df = df_movies.copy()

In [61]:
# selecionando as features de interesse
df = df[['object','text', 'likes']]
df.head(3)

,object,text,likes
0,007-cassino-royale-t23,Um dos melhores do 007,0
2489,007-operacao-skyfall-t31713,Javier Bardem caricato. Daniel Craig ruim como...,0
6141,10-coisas-que-eu-odeio-em-voce-t1964,ADORO!!,0


In [62]:
df.shape

(1736231, 3)

In [63]:
import time
start = time.time()

df['text'] = df['text'].str.lower()
df.head(3)

,object,text,likes
0,007-cassino-royale-t23,um dos melhores do 007,0
2489,007-operacao-skyfall-t31713,javier bardem caricato. daniel craig ruim como...,0
6141,10-coisas-que-eu-odeio-em-voce-t1964,adoro!!,0


In [64]:
def identify_tokens(row):
    review = row['text']
    tokens = nltk.word_tokenize(review)
    
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalnum()] # isalnum (não remove caracteres especiais e números) - isalpha (removeremos palavras / caracteres não alfanuméricos (como números e pontuação)
    return token_words

start = time.time()
df['text_tokenizado'] = df.apply(identify_tokens, axis=1)
print ("tempo gasto", (time.time() - start))

df['text_tokenizado'].head(3)

tempo gasto 1437.3517603874207


0                            [um, dos, melhores, do, 007]
2489    [javier, bardem, caricato, daniel, craig, ruim...
6141                                              [adoro]
Name: text_tokenizado, dtype: object

In [65]:
df.head(5)

,object,text,likes,text_tokenizado
0,007-cassino-royale-t23,um dos melhores do 007,0,"[um, dos, melhores, do, 007]"
2489,007-operacao-skyfall-t31713,javier bardem caricato. daniel craig ruim como...,0,"[javier, bardem, caricato, daniel, craig, ruim..."
6141,10-coisas-que-eu-odeio-em-voce-t1964,adoro!!,0,[adoro]
7108,12-anos-de-escravidao-t43448,existem certos filmes que funcionam como ápice...,2,"[existem, certos, filmes, que, funcionam, como..."
14078,127-horas-t24843,"o filme é do james franco, sem dúvidas. se não...",0,"[o, filme, é, do, james, franco, sem, dúvidas,..."


## Remoção de stop-words

In [81]:
import string

stp_words = nltk.corpus.stopwords.words('portuguese')
stop_words = stp_words +' '.join(string.punctuation).split(' ')+["``","''","“",'”']
print(stop_words)


['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

In [82]:
def remove_stops(row):
    my_list = row['text_tokenizado']
    #print('my_list: ', my_list)
    meaningful_words = [w for w in my_list if not w in stopwords]
    return (meaningful_words)


start = time.time()
df['text_stopwords'] = df.apply(remove_stops, axis=1)
print ("tempo gasto", (time.time() - start))

df[['text', 'text_tokenizado', 'text_stopwords']].head(3)

tempo gasto 367.84771251678467


,text,text_tokenizado,text_stopwords
0,um dos melhores do 007,"[um, dos, melhores, do, 007]","[melhores, 007]"
2489,javier bardem caricato. daniel craig ruim como...,"[javier, bardem, caricato, daniel, craig, ruim...","[javier, bardem, caricato, daniel, craig, ruim..."
6141,adoro!!,[adoro],[adoro]


# Salvando o resultado num dataframe

In [83]:
df.columns

Index(['object', 'text', 'likes', 'text_tokenizado', 'text_stopwords'], dtype='object')

In [84]:
# salvando num dataframe
# salvando o dataset num CSV     

df2 = df [['object', 'text_stopwords', 'likes']]

df2.to_csv(r"C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\df_util_tokenizado.csv",index=False)

In [85]:
# abrindo o arquivo TOKENIZADO
df_tokenizado = pd.read_csv(r"C:\Users\Herica\Documents\Herica\Pos\GitHub\base\NLP\df_util_tokenizado.csv")
df_tokenizado.head(3)

,object,text_stopwords,likes
0,007-cassino-royale-t23,"['melhores', '007']",0
1,007-operacao-skyfall-t31713,"['javier', 'bardem', 'caricato', 'daniel', 'cr...",0
2,10-coisas-que-eu-odeio-em-voce-t1964,['adoro'],0


In [86]:
df_tokenizado['likes'].unique()

array([  0,   2,   4,   1,   3,   5,   6,  29,  17,  27,  16,  10,  15,
         9,  11,   8,   7,  28,  42,  13,  14,  65,  31,  18,  21,  33,
        12,  23,  22,  66,  26,  20, 103,  19,  64,  53,  68,  46,  48,
        56,  40,  24,  55,  35,  32,  59, 123,  30,  25,  82,  38,  69,
        49, 122,  90,  83,  77,  36,  50,  43,  52,  47,  72,  41,  37,
        60,  73,  45,  44,  91,  34,  70,  67,  62,  75,  51,  88,  54,
        63, 124,  92,  76,  39,  58,  71,  80,  61, 127, 185, 138, 139,
        57, 109, 156,  79,  85, 179,  97,  74, 101, 217, 147,  96, 104,
       110,  89, 170, 151, 120,  84, 106, 190, 184, 100, 146, 172, 105,
       117, 261,  78,  99,  86, 132, 133, 112, 111, 118, 116, 157, 164,
       134, 137,  81, 187, 107,  98, 140,  94, 203, 102,  93, 166, 241,
       108, 273, 176, 136, 260, 180, 186, 183, 238, 113,  87, 267, 287,
       328, 163, 145, 255, 197, 175, 236, 353, 169, 128, 135,  95, 291,
       168, 126, 115, 131, 165, 234, 121, 329, 211, 148, 130, 11

# **2 - Visualização dos dados**
Coloque nesta seção os gráficos do PCA e do t-SNE, para cada representação. 

Responda também às seguintes perguntas: 

a) Existe algum padrão com relação às classes? 

b) Caso exista algum padrão, você pode concluir alguma coisa? 

c) Caso não exista, você consegue dizer se isso tem a ver com alguma representação ou classe?


## Divisão dos dados em treino e teste

In [87]:
from sklearn.model_selection import train_test_split

X = df_tokenizado['text_stopwords']
y = df_tokenizado['likes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [88]:
print ('X_train: ', X_train.shape, 'y_train', y_train.shape, 'X_test', X_test.shape, 'y_test', y_test.shape)

X_train:  (1388984,) y_train (1388984,) X_test (347247,) y_test (347247,)


## Representação TF-IDF
Você pode usar tanto o gensim quanto o scikit para montar esta representação, mas lembre-se que é importante fazer o pré-processamento dos textos.


In [89]:
from nltk.tokenize import word_tokenize


In [90]:
def preprocess(data:np.array=[], words:bool = False, toks:bool=False, sents:bool = False, feats:bool = False):
    dct_feats = {}
    arr_titles, arr_sents, arr_words, arr_titles_promt, arr_words_promt = [],[],[],[],[]
    qtd_parag, qtd_sent, qtd_words, mean_tok = [],[],[],[]

    for i in range(len(data)):
        
        if words or feats:
            # Cria a lista de palavras dos documentos
            tok = [tok.lower().replace("'","") for tok in word_tokenize(str(data[i]))\
                     if tok.lower() not in stp_words]
            arr_words.append(tok)             
            mean_tok.append(len(tok))
        
        if sents:
            # Cria a lista de senteças dos documentos
            for sent in sent_tokenize(str(data[i])):
                arr_sents.append(simple_preprocess(sent))
        
        if feats:
            counter = 0    # Inicializa o contador
            sp_parag = str(data[i]).split("     ")    # Separa o texto pelo recuo

            # Percorre o texto separado pelo recuo de parágrafo
            for parag in sp_parag:

                # Verifica se é um paragrafo de acordo com a primeira letra
                # Se a primeira letra é maiúscula conta como novo paragrafo
                if len(parag) > 0:
                    if parag[0].isupper():
                        counter += 1   
                        
            qtd_parag.append(counter / len(str(data[i])))    # Insere a quantidade de paragrafos
            qtd_sent.append(len(sent_tokenize(str(data[i]))) / len(str(data[i])))    # Insere a quantidade de sentenças 
            qtd_words.append(len(tok) / len(str(data[i])))    # Insere a quantidade de palavras

            dct_feats = dict(parag = qtd_parag, sents = qtd_sent, words = qtd_words)
        
    
    if toks: return mean_tok
    if sents: return arr_sents
    if words: return arr_words
    if feats: return dct_feats
    
    
toks = preprocess(df_tokenizado['text_stopwords'], words=True, toks=True)

In [91]:
# A representação abaixo é uma medida estatística calculada a partir da frequência de cada termo e tem o intuito de indicar a importância de cada palavra do documento em relação à coleção de documentos.


arr_words = preprocess(X_train, words=True)

dct_text = Dictionary(arr_words)

# Cria o 'bow' de cada documento
bow_text = [dct_text.doc2bow(line) for line in arr_words]

# Calcula a representação Tf-idf de cada documento
model_tfidf = TfidfModel(bow_text)

KeyError: 11

## Representação com o word2vec
O modelo poderá ser o apresentado na aula 03 ou algum outro modelo pré-treinado como os existentes no repositório http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc . Neste caso, cada documento deverá ser representado pelo vetor que resultar da média dos vetores de todas as palavras que o compõem. Em outras palavras, se D é composto pelas palavras w1, w2, …, wn, e seus vetores embeddings são v1, v2, …, vn, então a representação do documento de D será v = (v1 + v2 + … + vn) / n. 


## Extração de features do texto
Você deve pensar em ao menos 10 features para extrair do documento e que o possam representar. Aqui vão algumas sugestões: número de palavras, número de verbos, número de conjunções, número de palavras negativas, número de palavras fora do vocabulário, quantidades de entidades do tipo PESSOA, quantidade de entidades do tipo LOCAL, etc.


## Visualização dos seus dados com PCA

## Visualização dos seus dados com t-SNE

# **3 - Classificadores**
Descreva sucintamente os dois classificadores escolhidos (k-nn, SVM,RandomForest...). Você usou algum parâmetro que não seja padrão? Se sim, mencione nesta seção. 


# **4 - Resultados**
Escreva código que execute a validação cruzada em 5-folds para os dois classificadores escolhidos. Também responda às seguintes perguntas: Os embeddings realmente mostraram um resultado melhor que o TF-IDF? Se não, qual foi a representação que teve o melhor desempenho? A diferença foi muito grande?

# **5 - Conclusão**
Por fim fale aqui o que você conclui das visualizações e dos resultados. Tente explicar em detalhes por que um resultado, na sua opinião, foi melhor do que outro. Esta explicação pode incluir hipóteses para resultados melhores ou resultados piores. Também pode falar das dificuldades enfrentadas durante o trabalho e como conseguiu contorná-las.
